In [1]:
#Importing libraries. The first group is only needed to get the coordinates
from pyorbital import tlefile
from pyorbital.orbital import Orbital
from datetime import datetime, timedelta
import pandas as pd


import json

import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pandas_geojson import to_geojson

In [2]:
#Iniating the dataframpersonal access tokene. If you are processing multiple satellites, do not repeat this
df = pd.DataFrame(columns=['SatName','Time', 'longitude', 'latitude', 'altitude', 'px', 'py', 'pz'])

In [3]:
#Example satellite. TLEs are not needed if you want to use the latest. Then it will try to fetch the latest (not all work)

SatName="SUOMI100"
TLEline1="1 43804U 18099AY  23106.41328778  .00007811  00000-0  63443-3 0  9998"
TLEline2="2 43804  97.5788 170.4354 0014367 333.7649  26.2845 14.99802605238436"
#Replace the datetime with datetime.utcnow() if now is preferred
TimeStart=datetime(2022,1,6,3,0)
#This is the span of observation in seconds 
TimeSpanSec=3600

In [5]:
#If you want this to  try to fetch the lates TLEs, replace the line with orb = Orbital('NOAA-21')
#If you want to use TLElines in file orb = Orbital('SUOMI100', tle_file='/path/to/tle/files/tle.txt')
orb = Orbital(SatName, line1=TLEline1, line2=TLEline2)
orb.get_lonlatalt(TimeStart)

(-69.57985515265219, -8.003765620159887, 596.773648976479)

In [18]:
#If you want to change the delay between points, change the range increment value (here 5)

for x in range(0,TimeSpanSec,5):
    time=TimeStart+timedelta(seconds=x)

    ArrayP=orb.get_position(time)[1]
    ArrayG=orb.get_lonlatalt(time)

    df.loc[len(df.index)] = [SatName,time, ArrayG[0], ArrayG[1], ArrayG[2], ArrayP[0], ArrayP[1], ArrayP[2]] 
#Additionally an extra column is added as geojson is rather picky about the timestamp, so the timestamp is there as string. #flag make a prettier version
df['TimeString'] = df['Time'].astype(str)


ValueError: cannot set a row with mismatched columns

In [19]:
df

,SatName,Time,longitude,latitude,altitude,px,py,pz,TimeString
0,SUOMI100,2022-01-06 03:00:00,-69.579855,-8.003766,596.773649,0.010833,0.008015,0.069817,2022-01-06 03:00:00
1,SUOMI100,2022-01-06 03:00:05,-69.642436,-7.694229,596.657158,0.010774,0.007638,0.069869,2022-01-06 03:00:05
2,SUOMI100,2022-01-06 03:00:10,-69.704957,-7.384674,596.541535,0.010714,0.007260,0.069920,2022-01-06 03:00:10
3,SUOMI100,2022-01-06 03:00:15,-69.767422,-7.075102,596.426785,0.010654,0.006882,0.069968,2022-01-06 03:00:15
4,SUOMI100,2022-01-06 03:00:20,-69.829833,-6.765514,596.312915,0.010594,0.006503,0.070014,2022-01-06 03:00:20
...,...,...,...,...,...,...,...,...,...
2875,SUOMI100,2022-01-06 03:59:35,89.219434,-34.600675,604.564323,-0.000618,0.041155,-0.057929,2022-01-06 03:59:35
2876,SUOMI100,2022-01-06 03:59:40,89.138187,-34.907547,604.740499,-0.000535,0.041461,-0.057710,2022-01-06 03:59:40
2877,SUOMI100,2022-01-06 03:59:45,89.056491,-35.214362,604.916802,-0.000452,0.041766,-0.057489,2022-01-06 03:59:45
2878,SUOMI100,2022-01-06 03:59:50,88.974340,-35.521119,605.093219,-0.000369,0.042069,-0.057266,2022-01-06 03:59:50


In [10]:
#To save the dataframe as csv
df.to_csv('file_name.csv')

In [20]:
#To turn the dataframe to geojson feature collection filled with points
geo_jsonPoints = to_geojson(df, lat='latitude', lon='longitude',
                 properties=['SatName','TimeString','altitude'])
geo_jsonPoints

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'SatName': 'SUOMI100',
    'TimeString': '2022-01-06 03:00:00',
    'altitude': 596.773648976479},
   'geometry': {'type': 'Point',
    'coordinates': [-69.57985515265219, -8.003765620159887]}},
  {'type': 'Feature',
   'properties': {'SatName': 'SUOMI100',
    'TimeString': '2022-01-06 03:00:05',
    'altitude': 596.6571580272592},
   'geometry': {'type': 'Point',
    'coordinates': [-69.64243560539347, -7.694228815238179]}},
  {'type': 'Feature',
   'properties': {'SatName': 'SUOMI100',
    'TimeString': '2022-01-06 03:00:10',
    'altitude': 596.5415348356046},
   'geometry': {'type': 'Point',
    'coordinates': [-69.70495699473837, -7.384674295412272]}},
  {'type': 'Feature',
   'properties': {'SatName': 'SUOMI100',
    'TimeString': '2022-01-06 03:00:15',
    'altitude': 596.4267851611874},
   'geometry': {'type': 'Point',
    'coordinates': [-69.76742184672925, -7.075102436715624]}},
  {'type': 'Fea

In [21]:
from pandas_geojson import write_geojson
write_geojson(geo_jsonPoints, filename='geo_jsonPoints.geojson', indent=4)

In [ ]:
#Now geopandas part
#TBD